# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN, frecuentemente llamada también una Artificial Neural Network (ANN).  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven. 

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow.keras.datasets"**. 

In [84]:
import numpy as np
import tensorflow as tf

La siguiente instrucción, cuando se corre por primera vez, descarga el conjunto de datos en lo indicado por el parámetro path, relativo a  ~/.keras/datasets).  Como si se hubiera ejecutado Lo siguiente:

tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

luego separa los datos en un conjunto para entrenamiento y otro para pruebas.

Si se ejecuta más de una vez, ya no descarga el archivo.

In [85]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = tf.keras.datasets.mnist.load_data()
assert X_entreno.shape == (60000, 28, 28)
assert X_prueba.shape == (10000, 28, 28)
assert y_entreno.shape == (60000,)
assert y_prueba.shape == (10000,)

## Datos

Esta sección es donde pre-procesaremos nuestros datos.

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Lo haremos del mismo tamaño que el conjunto de prueba

In [86]:
num_obs_validacion = y_prueba.shape[0]

Usaremos una variable dedicada para el número de muestras de prueba

In [87]:
num_obs_prueba = y_prueba.shape[0]

Generalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [88]:
X_entreno_normalizado = X_entreno / 255

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [89]:
X_prueba_normalizado = X_prueba / 255

Una vez se han "normalizado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán 10000 para ser igual al conjunto de prueba.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [90]:
X_validacion = X_entreno_normalizado[-num_obs_validacion: , : , : ]
y_validacion = y_entreno[-num_obs_validacion:]

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [91]:
X_entreno = X_entreno_normalizado[ : X_entreno_normalizado.shape[0] - num_obs_validacion, : , : ]
y_entreno = y_entreno[ : y_entreno.shape[0] - num_obs_validacion]
num_obs_entreno = y_entreno.shape[0]

Convertir de Arreglos Numpy a Tensores

In [92]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

Barajear y hacer tandas con el conjunto de datos de entrenamiento

In [93]:
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)

Hacer tandas con los conjunto de validación y prueba, no se necesita barajearlos

In [94]:
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprenzaje profundo, casi siempre solo lo imaginamos.  Asi que esta vez, hagámoslo.  :)

In [95]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  No es una necesidad!

In [96]:
tamanio_capa_escondida = 250

# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28 píxeles, por lo tanto es un tensor de rango 2.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28 y lo convierte en  un vector (None), o (784,)...porque 28x28 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  output = activation(dot(entrada, peso) + sesgo).  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [97]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [98]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [99]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5


2024-08-08 18:46:32.487472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]
2024-08-08 18:46:32.487660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]


500/500 - 1s - loss: 0.2691 - accuracy: 0.9213 - val_loss: 0.1451 - val_accuracy: 0.9583 - 881ms/epoch - 2ms/step
Epoch 2/5


2024-08-08 18:46:33.266409: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


500/500 - 1s - loss: 0.1020 - accuracy: 0.9690 - val_loss: 0.0872 - val_accuracy: 0.9741 - 699ms/epoch - 1ms/step
Epoch 3/5
500/500 - 1s - loss: 0.0659 - accuracy: 0.9795 - val_loss: 0.0806 - val_accuracy: 0.9761 - 674ms/epoch - 1ms/step
Epoch 4/5
500/500 - 1s - loss: 0.0444 - accuracy: 0.9861 - val_loss: 0.0790 - val_accuracy: 0.9769 - 696ms/epoch - 1ms/step
Epoch 5/5
500/500 - 1s - loss: 0.0339 - accuracy: 0.9892 - val_loss: 0.0720 - val_accuracy: 0.9803 - 679ms/epoch - 1ms/step


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. Nunca debe probarse el modelo antes de haber completamente ajustado el modelo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [100]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 [==============================] - 0s 653us/step - loss: 11.8782 - accuracy: 0.9800


2024-08-08 18:46:36.140982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


In [101]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 11.88. Precisión de prueba: 98.00%


Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que puedan tener la oportunidad de mejorarla como ejercicio de laboratorio.

## Inciso 1

In [102]:
tamanio_capa_escondida = 200
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 1s - loss: 0.2928 - accuracy: 0.9155 - val_loss: 0.1402 - val_accuracy: 0.9591 - 855ms/epoch - 2ms/step
Epoch 2/5
500/500 - 1s - loss: 0.1133 - accuracy: 0.9665 - val_loss: 0.0936 - val_accuracy: 0.9733 - 687ms/epoch - 1ms/step
Epoch 3/5
500/500 - 1s - loss: 0.0723 - accuracy: 0.9780 - val_loss: 0.0923 - val_accuracy: 0.9738 - 624ms/epoch - 1ms/step
Epoch 4/5
500/500 - 1s - loss: 0.0521 - accuracy: 0.9831 - val_loss: 0.0846 - val_accuracy: 0.9747 - 612ms/epoch - 1ms/step
Epoch 5/5
500/500 - 1s - loss: 0.0392 - accuracy: 0.9873 - val_loss: 0.0956 - val_accuracy: 0.9722 - 569ms/epoch - 1ms/step
100/100 [==============================] - 0s 581us/step - loss: 16.0598 - accuracy: 0.9726
Pérdida de prueba: 16.06. Precisión de prueba: 97.26%


In [103]:
tamanio_capa_escondida = 250
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 1s - loss: 0.2680 - accuracy: 0.9218 - val_loss: 0.1322 - val_accuracy: 0.9615 - 887ms/epoch - 2ms/step
Epoch 2/5
500/500 - 1s - loss: 0.1017 - accuracy: 0.9691 - val_loss: 0.1001 - val_accuracy: 0.9699 - 755ms/epoch - 2ms/step
Epoch 3/5
500/500 - 1s - loss: 0.0669 - accuracy: 0.9791 - val_loss: 0.0988 - val_accuracy: 0.9698 - 710ms/epoch - 1ms/step
Epoch 4/5
500/500 - 1s - loss: 0.0471 - accuracy: 0.9851 - val_loss: 0.0769 - val_accuracy: 0.9771 - 699ms/epoch - 1ms/step
Epoch 5/5
500/500 - 1s - loss: 0.0364 - accuracy: 0.9883 - val_loss: 0.0825 - val_accuracy: 0.9766 - 709ms/epoch - 1ms/step
100/100 [==============================] - 0s 699us/step - loss: 13.7624 - accuracy: 0.9756
Pérdida de prueba: 13.76. Precisión de prueba: 97.56%


## Inciso 2

In [104]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


500/500 - 1s - loss: 0.4341 - accuracy: 0.8732 - val_loss: 0.1803 - val_accuracy: 0.9495 - 590ms/epoch - 1ms/step
Epoch 2/5
500/500 - 0s - loss: 0.1752 - accuracy: 0.9492 - val_loss: 0.1396 - val_accuracy: 0.9600 - 427ms/epoch - 853us/step
Epoch 3/5
500/500 - 0s - loss: 0.1317 - accuracy: 0.9611 - val_loss: 0.1251 - val_accuracy: 0.9640 - 384ms/epoch - 768us/step
Epoch 4/5
500/500 - 0s - loss: 0.1086 - accuracy: 0.9682 - val_loss: 0.1098 - val_accuracy: 0.9679 - 418ms/epoch - 837us/step
Epoch 5/5
500/500 - 0s - loss: 0.0895 - accuracy: 0.9728 - val_loss: 0.1091 - val_accuracy: 0.9680 - 377ms/epoch - 754us/step
100/100 [==============================] - 0s 409us/step - loss: 19.1642 - accuracy: 0.9657
Pérdida de prueba: 19.16. Precisión de prueba: 96.57%


## Inciso 3

In [105]:
tamanio_capa_escondida = 250
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4rta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5nta capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 1s - loss: 0.2676 - accuracy: 0.9192 - val_loss: 0.1333 - val_accuracy: 0.9596 - 1s/epoch - 3ms/step
Epoch 2/5
500/500 - 1s - loss: 0.1075 - accuracy: 0.9671 - val_loss: 0.0894 - val_accuracy: 0.9719 - 1s/epoch - 2ms/step
Epoch 3/5
500/500 - 1s - loss: 0.0732 - accuracy: 0.9777 - val_loss: 0.1071 - val_accuracy: 0.9711 - 1s/epoch - 3ms/step
Epoch 4/5
500/500 - 1s - loss: 0.0588 - accuracy: 0.9823 - val_loss: 0.1009 - val_accuracy: 0.9717 - 1s/epoch - 2ms/step
Epoch 5/5
500/500 - 1s - loss: 0.0468 - accuracy: 0.9855 - val_loss: 0.1039 - val_accuracy: 0.9721 - 1s/epoch - 2ms/step
100/100 [==============================] - 0s 940us/step - loss: 24.0427 - accuracy: 0.9699
Pérdida de prueba: 24.04. Precisión de prueba: 96.99%


## Inciso 4

In [106]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


500/500 - 1s - loss: 1.0307 - accuracy: 0.7793 - val_loss: 0.4111 - val_accuracy: 0.9069 - 515ms/epoch - 1ms/step
Epoch 2/5
500/500 - 0s - loss: 0.3462 - accuracy: 0.9097 - val_loss: 0.2625 - val_accuracy: 0.9285 - 353ms/epoch - 706us/step
Epoch 3/5
500/500 - 0s - loss: 0.2526 - accuracy: 0.9298 - val_loss: 0.2137 - val_accuracy: 0.9407 - 364ms/epoch - 728us/step
Epoch 4/5
500/500 - 0s - loss: 0.2070 - accuracy: 0.9408 - val_loss: 0.1804 - val_accuracy: 0.9491 - 356ms/epoch - 713us/step
Epoch 5/5
500/500 - 0s - loss: 0.1776 - accuracy: 0.9491 - val_loss: 0.1640 - val_accuracy: 0.9543 - 356ms/epoch - 712us/step
100/100 [==============================] - 0s 413us/step - loss: 0.1906 - accuracy: 0.9420
Pérdida de prueba: 0.19. Precisión de prueba: 94.20%


## Inciso 5

In [107]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 0s - loss: 0.4321 - accuracy: 0.8792 - val_loss: 0.1976 - val_accuracy: 0.9435 - 493ms/epoch - 987us/step
Epoch 2/5
500/500 - 0s - loss: 0.1791 - accuracy: 0.9469 - val_loss: 0.1401 - val_accuracy: 0.9601 - 335ms/epoch - 670us/step
Epoch 3/5
500/500 - 0s - loss: 0.1291 - accuracy: 0.9620 - val_loss: 0.1231 - val_accuracy: 0.9644 - 336ms/epoch - 672us/step
Epoch 4/5
500/500 - 0s - loss: 0.1021 - accuracy: 0.9694 - val_loss: 0.1081 - val_accuracy: 0.9681 - 335ms/epoch - 671us/step
Epoch 5/5
500/500 - 0s - loss: 0.0840 - accuracy: 0.9744 - val_loss: 0.1005 - val_accuracy: 0.9699 - 335ms/epoch - 670us/step
100/100 [==============================] - 0s 390us/step - loss: 0.1338 - accuracy: 0.9622
Pérdida de prueba: 0.13. Precisión de prueba: 96.22%


## Inciso 6

In [111]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 10000
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


2024-08-08 18:51:40.490231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]
2024-08-08 18:51:40.490910: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]


5/5 - 0s - loss: 2.2346 - accuracy: 0.1852 - val_loss: 2.0920 - val_accuracy: 0.3159 - 413ms/epoch - 83ms/step
Epoch 2/5


2024-08-08 18:51:40.850578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


5/5 - 0s - loss: 2.0082 - accuracy: 0.3630 - val_loss: 1.8316 - val_accuracy: 0.4591 - 215ms/epoch - 43ms/step
Epoch 3/5
5/5 - 0s - loss: 1.7384 - accuracy: 0.4952 - val_loss: 1.5260 - val_accuracy: 0.6213 - 164ms/epoch - 33ms/step
Epoch 4/5
5/5 - 0s - loss: 1.4395 - accuracy: 0.6463 - val_loss: 1.2286 - val_accuracy: 0.7359 - 168ms/epoch - 34ms/step
Epoch 5/5
5/5 - 0s - loss: 1.1610 - accuracy: 0.7375 - val_loss: 0.9673 - val_accuracy: 0.7959 - 169ms/epoch - 34ms/step
1/1 [==============================] - 0s 69ms/step - loss: 36.9337 - accuracy: 0.7850
Pérdida de prueba: 36.93. Precisión de prueba: 78.50%


2024-08-08 18:51:41.663325: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


## Inciso 7

In [112]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 1
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


2024-08-08 18:52:03.226305: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [50000,28,28]
	 [[{{node Placeholder/_0}}]]
2024-08-08 18:52:03.226612: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]
2024-08-08 18:52:20.410848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape

50000/50000 - 19s - loss: 0.2580 - accuracy: 0.9247 - val_loss: 0.1469 - val_accuracy: 0.9577 - 19s/epoch - 388us/step
Epoch 2/5
50000/50000 - 19s - loss: 0.1597 - accuracy: 0.9553 - val_loss: 0.1880 - val_accuracy: 0.9541 - 19s/epoch - 384us/step
Epoch 3/5
50000/50000 - 19s - loss: 0.1434 - accuracy: 0.9614 - val_loss: 0.1624 - val_accuracy: 0.9619 - 19s/epoch - 384us/step
Epoch 4/5
50000/50000 - 19s - loss: 0.1317 - accuracy: 0.9649 - val_loss: 0.1692 - val_accuracy: 0.9612 - 19s/epoch - 378us/step
Epoch 5/5
50000/50000 - 19s - loss: 0.1254 - accuracy: 0.9683 - val_loss: 0.1960 - val_accuracy: 0.9640 - 19s/epoch - 383us/step
  209/10000 [..............................] - ETA: 2s - loss: 25.5792 - accuracy: 0.9665      

2024-08-08 18:53:39.142339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


10000/10000 [==============================] - 3s 242us/step - loss: 52.1039 - accuracy: 0.9597
Pérdida de prueba: 52.10. Precisión de prueba: 95.97%


In [113]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 1
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


2024-08-08 18:56:11.163840: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]
2024-08-08 18:56:11.164445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]
2024-08-08 18:56:23.811729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000

50000/50000 - 15s - loss: 0.2750 - accuracy: 0.9157 - val_loss: 0.1683 - val_accuracy: 0.9456 - 15s/epoch - 298us/step
Epoch 2/5
50000/50000 - 15s - loss: 0.1405 - accuracy: 0.9575 - val_loss: 0.1174 - val_accuracy: 0.9659 - 15s/epoch - 305us/step
Epoch 3/5
50000/50000 - 16s - loss: 0.1097 - accuracy: 0.9667 - val_loss: 0.1305 - val_accuracy: 0.9620 - 16s/epoch - 314us/step
Epoch 4/5
50000/50000 - 15s - loss: 0.0888 - accuracy: 0.9730 - val_loss: 0.1250 - val_accuracy: 0.9662 - 15s/epoch - 304us/step
Epoch 5/5
50000/50000 - 15s - loss: 0.0807 - accuracy: 0.9750 - val_loss: 0.1256 - val_accuracy: 0.9657 - 15s/epoch - 293us/step
  389/10000 [>.............................] - ETA: 2s - loss: 18.9872 - accuracy: 0.9563    

2024-08-08 18:57:26.918735: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


10000/10000 [==============================] - 3s 249us/step - loss: 27.2217 - accuracy: 0.9546
Pérdida de prueba: 27.22. Precisión de prueba: 95.46%


## Inciso 8

In [114]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

learning_rate = 0.0001
optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compilar el modelo con el optimizador personalizado
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


2024-08-08 18:57:58.901745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]
2024-08-08 18:57:58.902136: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]


500/500 - 1s - loss: 1.3447 - accuracy: 0.6421 - val_loss: 0.5680 - val_accuracy: 0.8710 - 592ms/epoch - 1ms/step
Epoch 2/5


2024-08-08 18:57:59.398596: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


500/500 - 0s - loss: 0.4636 - accuracy: 0.8813 - val_loss: 0.3493 - val_accuracy: 0.9070 - 387ms/epoch - 774us/step
Epoch 3/5
500/500 - 0s - loss: 0.3484 - accuracy: 0.9029 - val_loss: 0.2946 - val_accuracy: 0.9196 - 362ms/epoch - 724us/step
Epoch 4/5
500/500 - 0s - loss: 0.3046 - accuracy: 0.9139 - val_loss: 0.2665 - val_accuracy: 0.9270 - 369ms/epoch - 739us/step
Epoch 5/5
500/500 - 0s - loss: 0.2786 - accuracy: 0.9207 - val_loss: 0.2477 - val_accuracy: 0.9314 - 422ms/epoch - 845us/step
100/100 [==============================] - 0s 454us/step - loss: 32.5586 - accuracy: 0.9243
Pérdida de prueba: 32.56. Precisión de prueba: 92.43%


2024-08-08 18:58:01.053078: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


## Inciso 9

In [ ]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

learning_rate = 0.02
optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compilar el modelo con el optimizador personalizado
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 0s - loss: 0.3210 - accuracy: 0.9022 - val_loss: 0.1918 - val_accuracy: 0.9456 - 499ms/epoch - 999us/step
Epoch 2/5
500/500 - 0s - loss: 0.2064 - accuracy: 0.9418 - val_loss: 0.2168 - val_accuracy: 0.9466 - 342ms/epoch - 684us/step
Epoch 3/5
500/500 - 0s - loss: 0.1807 - accuracy: 0.9496 - val_loss: 0.1803 - val_accuracy: 0.9536 - 338ms/epoch - 676us/step
Epoch 4/5
500/500 - 0s - loss: 0.1813 - accuracy: 0.9507 - val_loss: 0.1897 - val_accuracy: 0.9499 - 369ms/epoch - 739us/step
Epoch 5/5
500/500 - 0s - loss: 0.1560 - accuracy: 0.9580 - val_loss: 0.1996 - val_accuracy: 0.9517 - 359ms/epoch - 718us/step
100/100 [==============================] - 0s 397us/step - loss: 42.9773 - accuracy: 0.9447
Pérdida de prueba: 42.98. Precisión de prueba: 94.47%


## Inciso 10

In [ ]:
# Definir los datos
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 150  # Aumentar el tamaño del lote
datos_entreno = datos_entreno.shuffle(buffer_size=num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 250


modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4rta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5nta capa escondida

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 6xta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 7ima capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
learning_rate = 0.003
optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compilar el modelo con el optimizador personalizado
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


2024-08-08 18:45:47.656893: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]
2024-08-08 18:45:47.657271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [50000]
	 [[{{node Placeholder/_1}}]]


334/334 - 2s - loss: 0.3284 - accuracy: 0.8988 - val_loss: 0.1427 - val_accuracy: 0.9612 - 2s/epoch - 5ms/step
Epoch 2/5


2024-08-08 18:45:49.050857: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


334/334 - 1s - loss: 0.1373 - accuracy: 0.9620 - val_loss: 0.1293 - val_accuracy: 0.9665 - 1s/epoch - 4ms/step
Epoch 3/5
334/334 - 1s - loss: 0.1055 - accuracy: 0.9714 - val_loss: 0.1074 - val_accuracy: 0.9713 - 1s/epoch - 4ms/step
Epoch 4/5
334/334 - 1s - loss: 0.0941 - accuracy: 0.9752 - val_loss: 0.1150 - val_accuracy: 0.9696 - 1s/epoch - 4ms/step
Epoch 5/5
334/334 - 1s - loss: 0.0785 - accuracy: 0.9792 - val_loss: 0.1274 - val_accuracy: 0.9732 - 1s/epoch - 4ms/step
67/67 [==============================] - 0s 1ms/step - loss: 23.0045 - accuracy: 0.9704
Pérdida de prueba: 23.00. Precisión de prueba: 97.04%


2024-08-08 18:45:54.374771: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]
